In [4]:
# 1. EDA: Sanity check splits and class balance
import pandas as pd

df = pd.read_csv('dataset/UNIFIED_ALL_SPLIT.csv')

print('Split counts:')
print(df['split'].value_counts(), '\n')

print('Per split language distribution:')
print(df.groupby('split')['language'].value_counts(), '\n')

print('is_hate by split:')
*** End Patch
        running_loss = 0.0
        for batch in train_loader:
            batch = move_batch_to_device(batch)
            optimizer.zero_grad()
            logits = model(batch['input_ids'], batch['attention_mask'])
            targets = {k: batch[k] for k in ['hate_type', 'target_group', 'severity']}
            masks = {k: batch[f'{k}_mask'] for k in targets.keys()}
            loss = multitask_loss(*logits, targets, masks, config['task_weights'])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
            optimizer.step()
            scheduler.step()
            running_loss += loss.item()

        train_loss = running_loss / max(1, len(train_loader))
        val_metrics = evaluate(model, val_loader, config['task_weights'])
        val_loss = val_metrics['loss']
        epoch_time = time.time() - start

        log_payload = {
            'epoch': epoch,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'epoch_time': epoch_time,
            **val_metrics
        }
        history.append(log_payload)

        if use_wandb:
            wandb.log(log_payload)

        print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, time={epoch_time:.1f}s")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_ckpt_path)
            print(f"  New best checkpoint saved to {best_ckpt_path}")
        else:
            patience_counter += 1
            if patience_counter >= config['patience']:
                print('Early stopping triggered.')
                break

    if use_wandb:
        wandb.finish()

    return best_ckpt_path, history



IndentationError: unindent does not match any outer indentation level (<string>, line 58)

In [ ]:
# 9. Optional: create smaller loaders for smoke testing/troubleshooting
SMOKE_TRAIN_SIZE = 512
SMOKE_VAL_SIZE = 256
SMOKE_TEST_SIZE = 256

smoke_train_df = train_df.sample(n=SMOKE_TRAIN_SIZE, random_state=SEED)
smoke_val_df = val_df.sample(n=SMOKE_VAL_SIZE, random_state=SEED)
smoke_test_df = test_df.sample(n=SMOKE_TEST_SIZE, random_state=SEED)

smoke_train_loader = DataLoader(HateDataset(smoke_train_df, tokenizer, max_length=MAX_LENGTH), batch_size=8, shuffle=True)
smoke_val_loader = DataLoader(HateDataset(smoke_val_df, tokenizer, max_length=MAX_LENGTH), batch_size=8, shuffle=False)
smoke_test_loader = DataLoader(HateDataset(smoke_test_df, tokenizer, max_length=MAX_LENGTH), batch_size=8, shuffle=False)

len(smoke_train_df), len(smoke_val_df), len(smoke_test_df)


NameError: name 'train_df' is not defined

In [ ]:
# 10. Run a quick smoke-test training (set use_wandb=True for tracking)
quick_config = training_config.copy()
quick_config.update({'epochs': 1, 'patience': 1})

best_checkpoint, history = train_model(smoke_train_loader, smoke_val_loader, config=quick_config, run_name='xlmr_smoke', use_wandb=False)
history


In [ ]:
# 11. Evaluate trained checkpoint on smoke val/test loaders
best_model = MultiTaskXLMRRoberta().to(device)
best_model.load_state_dict(torch.load(best_checkpoint, map_location=device))
val_results = evaluate(best_model, smoke_val_loader)
test_results = evaluate(best_model, smoke_test_loader)
print('Validation metrics:', val_results)
print('Test metrics:', test_results)


Validation metrics: {'loss': np.float64(1.1306458543986082), 'hate_type_macro_f1': 0.1103448275862069, 'hate_type_micro_f1': 0.38095238095238093, 'target_group_macro_f1': 0.16666666666666666, 'target_group_micro_f1': 0.5, 'severity_macro_f1': 0.24709564925106617, 'severity_micro_f1': 0.53515625}
Test metrics: {'loss': np.float64(1.2015934903174639), 'hate_type_macro_f1': 0.09795918367346938, 'hate_type_micro_f1': 0.32432432432432434, 'target_group_macro_f1': 0.23899371069182387, 'target_group_micro_f1': 0.5588235294117647, 'severity_macro_f1': 0.2608990900448187, 'severity_micro_f1': 0.578125}


In [ ]:
# 12. W&B Login (run this first to authenticate)
try:
    import wandb
except ModuleNotFoundError:
    import sys, subprocess
    print("'wandb' not found. Installing via pip...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "wandb"])
    import wandb
    print("'wandb' installed successfully.")

# Option 1: Interactive login (uncomment and run)
# wandb.login()

# Option 2…
wandb.login(key="61dd3d59137a5043373cd8ecc8f74c4d1c620ea6")

# For now, we'll try to use existing login or prompt if needed
try:
    # Check if already logged in
    api = wandb.Api()
    print("W&B login successful or already logged in")
except Exception as e:
    print("W&B not logged in. Please authenticate:")
    print("Option 1: Run 'wandb.login()' interactively")
    print("Option 2: Set environment variable WANDB_API_KEY")
    print("Option 3: Get your API key from: https://wandb.ai/authorize")
    print(f"Error: {e}")

# 13. Full training with full dataset and W&B tracking
# Update training config for full run
full_training_config = training_config.copy()
full_training_config.update({
    'epochs': 5,  # Increase epochs for full training
    'patience': 3,  # More patience for full training
})

print("Starting full training with xlm-roberta-large on full dataset...")
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")
print(f"Device: {device}")

best_checkpoint_full, history_full = train_model(
    train_loader, 
    val_loader, 
    config=full_training_config, 
    run_name='xlmr_full_large', 
    use_wandb=True  # Enable W&B tracking
)
history_full



wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Currently logged in as: c221025 (c221025-international-islamic-university-chittagong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B login successful or already logged in


NameError: name 'training_config' is not defined

In [ ]:
# 14. Evaluate best checkpoint on full validation and test splits
# Make sure to run cell 11 (training) first!
import wandb

# Check if training has been completed
try:
    # Verify variables exist from training cell
    if 'best_checkpoint_full' not in locals() and 'best_checkpoint_full' not in globals():
        raise NameError("best_checkpoint_full not found. Please run cell 11 (training) first!")
    if 'full_training_config' not in locals() and 'full_training_config' not in globals():
        raise NameError("full_training_config not found. Please run cell 11 (training) first!")
    
    print("Loading best checkpoint and evaluating on full val/test sets...")
    best_model_full = MultiTaskXLMRRoberta().to(device)
    best_model_full.load_state_dict(torch.load(best_checkpoint_full, map_location=device))

    print("\n=== Validation Set Results ===")
    val_results_full = evaluate(best_model_full, val_loader, full_training_config['task_weights'])
    for key, value in val_results_full.items():
        if value is not None:
            print(f"{key}: {value:.4f}")

    print("\n=== Test Set Results ===")
    test_results_full = evaluate(best_model_full, test_loader, full_training_config['task_weights'])
    for key, value in test_results_full.items():
        if value is not None:
            print(f"{key}: {value:.4f}")

    # Log final metrics to W&B if enabled
    try:
        if wandb.run is not None:
            wandb.log({
                'final_val_loss': val_results_full['loss'],
                'final_test_loss': test_results_full['loss'],
                **{f'final_val_{k}': v for k, v in val_results_full.items() if k != 'loss' and v is not None},
                **{f'final_test_{k}': v for k, v in test_results_full.items() if k != 'loss' and v is not None}
            })
            wandb.finish()
    except Exception as wb_error:
        print(f"\nNote: Could not log to W&B: {wb_error}")

    print("\n=== Summary ===")
    print(f"Best checkpoint saved at: {best_checkpoint_full}")
    print("\nFull results dictionaries:")
    print("Validation:", val_results_full)
    print("Test:", test_results_full)
    
except NameError as e:
    print(f"Error: {e}")
    print("\nPlease run cell 11 (Full training) first before running this evaluation cell.")
except FileNotFoundError as e:
    print(f"Error: Checkpoint file not found: {e}")
    print("Please ensure cell 11 (Full training) completed successfully.")
except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()



ModuleNotFoundError: No module named 'wandb'